In [2]:
import pandas as pd

# Creating a panda dataframe to display:  
- tmID: String  
- year: Year  
- Wins_agg: total wins / total players  
- Losses_agg: total losses / total players  
- GP_agg: total games played / total players  
- Mins_over_GA_agg: total minutes played / total goals against  
- GA_over_SA_agg: total goals against / total shots against  
- avg_percentage_wins: calculate the percentage of games won for each player, then take the mean at team leve

In [3]:
#read goaliedata_clean file after parsing file, specifying which columns to load
data = pd.read_csv('goaliedata_clean.csv', 
                   usecols = ['playerID','year','tmID','GP','Min','W','L', 'T/OL','ENG','SHO','GA','SA']
                  )

__Winning Percentage is the only player specfic stat, so it will be calculated first__   
__Grouping by year and playerID allows Winning Percentage to be calculated__

In [4]:
#Group by player and year and get sums of wins and game played
WP_df = (data.groupby(['year','playerID'])
         .agg({'W':'sum', #Total wins
               'GP':'sum' #Total games played
              })
        )
#Add winning percentage column
WP_df["WinningPrecentage"] = WP_df['W']/WP_df['GP']

#change names to reduce ambiguity between columns after join
WP_df = (WP_df
         .reset_index()
         .rename(columns={'year':'year_WP_df',
                          'playerID':'playerID_WP_df',
                          'W':'W_WP_df',
                          'GP':'GP_WP_df'
                         }
                )
        )

In [6]:
#Join data with the WP_DF to get winning precentage in main df
df = (pd.merge(data, 
              WP_df,
              how='left',
              left_on=['year', 'playerID'],
              right_on=['year_WP_df', 'playerID_WP_df']
              )
      .drop(columns={'year_WP_df', 'playerID_WP_df', 'W_WP_df', 'GP_WP_df'})
     )

__The remaining stats are team specfic__  
__Therefore grouping by year and tmID allows__

In [12]:
#when grouping by year and team, get total wins, total loses, total players pleyed, total game played, total minutes played, total goals against
df = (df
      .groupby(['year','tmID'])
      .agg({'W': 'sum', #Total wins
            'L': 'sum', #Total losses
            'playerID': 'nunique', #Number of unique players
            'GP' : 'sum', #Total games played
            'Min' : 'sum', #Total minutes
            'GA' : 'sum', #Total goals against
            'SA' : 'sum', #Total shots against
            'WinningPrecentage' : 'mean' #Average winning precentage
           }
          )
      .rename(columns={'W':'TotalWins',
                       'L':'TotalLosses',
                       'playerID':'TotalPlayers',
                       'GP':'TotalGamesPlayed',
                       'Min':'TotalMinutes',
                       'GA':'TotalGoalsAgainst',
                       'SA':'TotalShotsAgainst',
                       'WinningPrecentage':'avg_percentage_wins'
                      }
             )
      .reset_index()
     )

In [13]:
#Create all the required columns for the final df
df['Wins_agg'] = df['TotalWins']/df['TotalPlayers']
df['Losses_agg'] = df['TotalLosses']/df['TotalPlayers']
df['GP_agg'] = df['TotalGamesPlayed']/df['TotalPlayers']
df['Mins_over_GA_agg'] = df['TotalMinutes']/df['TotalGoalsAgainst']
df['GA_over_SA_agg'] = df['TotalGoalsAgainst']/df['TotalShotsAgainst']

In [15]:
#Select required columns
df = df[["tmID", "year", "Wins_agg","Losses_agg", "GP_agg", "Mins_over_GA_agg","GA_over_SA_agg","avg_percentage_wins"]]

In [17]:
#Viewing data
df.tail()

,tmID,year,Wins_agg,Losses_agg,GP_agg,Mins_over_GA_agg,GA_over_SA_agg,avg_percentage_wins
1485,TBL,2010,11.500000,6.250000,22.250000,22.000000,0.096719,0.628157
1486,TOR,2010,12.333333,11.333333,31.000000,20.995763,0.093244,0.378248
1487,VAN,2010,27.000000,9.500000,42.500000,28.033898,0.071747,0.636667
1488,WAS,2010,16.000000,7.666667,29.666667,26.236842,0.080034,0.561398
1489,STL,2011,0.000000,0.000000,0.000000,NaN,NaN,NaN
